<a href="https://colab.research.google.com/github/delhian/recomender_systems/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Коллаборативная фильтрация

В предыдущей модели (content-based рекомендации) мы не учитывали поведение пользователей, так как смотрели только на похожесть фильмов между собой.  

Коллаборативная фильтрация работает лучше, так как мы можем использовать предыдущий подход и применить его к пользователям: собрать всех пользователей в одну матрицу и посчитать, какие пользователи похожи между собой.

Звучит отлично, но есть проблема: когда к нам приходят новые пользователи, у нас нет информации о том, на кого они похожи. Это называется проблемой холодного старта и ее можно решать разными способами:

1. Самым простым решением будет использовать popularity-based подход, при котором пользовалю просто предлагают самые популярные фильмы среди всех. Мы уже знаем, что это далеко не оптимальный подход, но его можно улучшить. Например, можно дополнительно использовать информацию, которую пользователь дает при регистрации (пол, возраст) или определять географический регион и советовать топ фильмов только оттуда.
2. Следующим шагом можно добавлять рекомендации из отдельной content-based рекомендательной системы. Когда пользователь уже посмотрел какие-то фильмы, информации для коллаборативной фильтрации все еще мало. Несмотря на это, мы можем давать пользователю рекомендации на основе тех фильмов, которые он уже посмотрел.

## 1.1. Memory-based подход

Есть несколько способов применения коллаборативной фильтрации.  
Один из них – подход, основанный на памяти. Этот подход подразумевает, что мы не используем никаких моделей для расчетов рекомендаций: все необходимые данные мы храним в одной таблице и они все должны находиться в памяти для создания рекомендации.

В данном подходе мы можем составить матрицу из действий пользователей, а затем сравнить пользователей с помощью какой-нибудь функции. Например, все той же косинусной метрике.

Недостатком такого метода является то, что при добавлении нового пользователя нам нужно заново строить матрицу. При этом, матрица будет разреженная (так как пользователей и фильмов может быть много), поэтому вычисления с ней будут более трудные.

Заново подготовим необходимые данные и функции:

In [ ]:
#@title
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from collections import defaultdict

NUM_USERS = 10_000
NUM_ITEMS = 1_000
USER_COL = 'user_id'
ITEM_COL = 'item_id'
RATING_COL = 'rating'
user_id = np.arange(start = 0, stop = NUM_USERS)
item_id = np.arange(start = 0, stop = NUM_ITEMS)
np.random.seed(42)

user_item_dict = defaultdict(list)
genres = ['Action','Comedy','Drama','Fantasy','Horror','Mystery','Romance','Thriller','Western']
for id in user_id:
    num_rand_item = np.random.randint(low = 3, high = 5)
    rand_items = np.random.choice(item_id, size = num_rand_item, replace = False)
    rand_rating = np.random.randint(low = 1, high = 10, size = num_rand_item)

    for uid, iid,rating in zip([id] * num_rand_item, rand_items, rand_rating):
        user_item_dict['user_id'].append(uid)
        user_item_dict['item_id'].append(iid)
        user_item_dict['rating'].append(rating)

ratings = pd.DataFrame(user_item_dict)
ratings[['user_id','item_id']] = ratings[['user_id','item_id']].astype(str)

item_genre_dict = defaultdict(list)
for iid in item_id:
    num_rand_genre = np.random.randint(low = 1, high = 3)
    rand_genres = np.random.choice(genres, size = num_rand_genre, replace = False)
    item_genre_dict['item_id'].append(iid)
    item_genre_dict['genres'].append(', '.join(list(rand_genres)))

items = pd.DataFrame(item_genre_dict)
items = items.astype(str)

def top_k_items(item_id, top_k, corr_mat, map_name):
    
    # sort correlation value ascendingly and select top_k item_id
    top_items = corr_mat[item_id,:].argsort()[-top_k:][::-1] 
    top_items = [map_name[e] for e in top_items] 

    return top_items

rated_items = items.loc[items[ITEM_COL].isin(ratings[ITEM_COL])].copy()
genre = rated_items['genres'].str.split(",", expand=True)

all_genre = set()
for c in genre.columns:
    distinct_genre = genre[c].str.lower().str.strip().unique()
    all_genre.update(distinct_genre)
all_genre.remove(None)

item_genre_mat = rated_items[[ITEM_COL, 'genres']].copy()
item_genre_mat['genres'] = item_genre_mat['genres'].str.lower().str.strip()

for genre in all_genre:
    item_genre_mat[genre] = np.where(item_genre_mat['genres'].str.contains(genre), 1, 0)
item_genre_mat = item_genre_mat.drop(['genres'], axis=1)
item_genre_mat = item_genre_mat.set_index(ITEM_COL)

ind2name = {ind:name for ind, name in enumerate(item_genre_mat.index)}
name2ind = {v:k for k, v in ind2name.items()}

In [ ]:
from scipy.sparse import csr_matrix

row = ratings[USER_COL]
col = ratings[ITEM_COL]
data = ratings[RATING_COL]

mat = csr_matrix((data, (row, col)), shape=(NUM_USERS, NUM_ITEMS))
mat.eliminate_zeros()

sparsity = float(len(mat.nonzero()[0]))
sparsity /= (mat.shape[0] * mat.shape[1])
sparsity *= 100
print(f'Sparsity: {sparsity:4.2f}%. This means that {sparsity:4.2f}% of the user-item ratings have a value.')

Sparsity: 0.35%. This means that 0.35% of the user-item ratings have a value.


In [ ]:
item_corr_mat = cosine_similarity(mat.T)

print("\nThe top-k similar movie to item_id 99")
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)

display(items.loc[items[ITEM_COL].isin(similar_items)])


The top-k similar movie to item_id 99


,item_id,genres
99,99,"Romance, Action"
248,248,Horror
352,352,"Romance, Action"
392,392,"Romance, Action"
507,507,"Horror, Western"
570,570,Horror
730,730,Fantasy
757,757,"Romance, Thriller"
824,824,Fantasy
899,899,Comedy


## 1.2. Model-based подход

Мы уже рассмотрели недостаток memory-based подхода: все приходится держать в памяти, поддерживать такую систему сложно.  
Model-based подход заключается в построении модели, которая будет заниматься предсказанием. Алгоритмов, основанных на model-based подходе, много. Мы рассмотрим некоторые из них.

### 1.2.1. Truncated SVD

Многие подходы так или иначе используют преобразования матриц (например, разложение или уменьшение размерности). Это помогает сократить вычисления и хранить меньше информации в памяти. 

В данном случае мы будем использовать TruncatedSVD в качестве алгоритма уменьшения размерности. SVD (Singular Value Decomposition) позволяет уменьшить размерность матрицы и эффективно работает с разреженными матрицами – у нас как раз такая.

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_latent_factors = 10

item_svd = TruncatedSVD(n_components = n_latent_factors)
item_features = item_svd.fit_transform(mat.transpose()) 

item_corr_mat = cosine_similarity(item_features)

print("\nThe top-k similar movie to item_id 99")
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)

display(items.loc[items[ITEM_COL].isin(similar_items)])


The top-k similar movie to item_id 99


,item_id,genres
99,99,"Romance, Action"
346,346,"Drama, Action"
352,352,"Romance, Action"
359,359,"Horror, Mystery"
392,392,"Romance, Action"
409,409,Mystery
797,797,"Western, Drama"
885,885,Romance
903,903,Thriller
999,999,Fantasy


Что нужно запомнить:
1. Мы уменьшили размер матрицы, но при росте количества пользователей/фильмов размер матрицы неизбежно будет расти
2. Уменьшенная матрица перестала быть интерпретируемой: до этого мы по матрице могли понять, от чего зависит та или иная рекомендация. Теперь эта возможность пропала – SVD перевел данные в другое пространство

### 1.2.2 Funk MF

Другой подход использует алгоритм Funk MF, названный в честь Саймона Фанка, который описал алгоритм в 2006 году, когда проходило соревнование Netflix по рекомендательным системам.  

Этот алгоритм раскладывает матрицу user-item на две матрицы с меньшей размерностью. При этом, первая матрица имеет строку для каждого пользователя, а вторая – столбец для каждого объекта (например, фильма). Векторы (строки) в этой матрице называются латентными векторами, а предсказание рейтинга можно сделать, перемножив эти матрицы (или соответствующие вектор и столбец)

Мы возьмем готовую реализацию алгоритма Funk MF из библиотеки [Surprise](https://surprise.readthedocs.io/en/stable/).

In [ ]:
!pip install scikit-surprise -q

     |████████████████████████████████| 11.8 MB 6.3 MB/s 


In [ ]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split

def pred2dict(predictions, top_k=None):
    rec_dict = defaultdict(list)
    for user_id, item_id, actual_rating, pred_rating, _ in predictions:
        rec_dict[user_id].append((item_id, pred_rating))        
        
    return rec_dict

def get_top_k_recommendation(rec_dict, user_id, top_k, ind2name):
    pred_ratings = rec_dict[user_id]
    pred_ratings = sorted(pred_ratings, key=lambda x: x[1], reverse=True)
    pred_ratings = pred_ratings[:top_k]
    recs = [ind2name[e[0]] for e in pred_ratings]
    
    return recs

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings, reader)
train, test = train_test_split(data, test_size=.2, random_state=42)

In [ ]:
algo = SVD(random_state = 42)
algo.fit(train)
pred = algo.test(test);

accuracy.rmse(pred)

RMSE: 2.6256


2.6256196381569668

In [ ]:
# extract the item features from algorithm
item_corr_mat = cosine_similarity(algo.qi)

print("\nThe top-k similar movie to item_id 99")
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)

display(items.loc[items[ITEM_COL].isin(similar_items)])


The top-k similar movie to item_id 99


,item_id,genres
99,99,"Romance, Action"
114,114,"Thriller, Romance"
180,180,Comedy
194,194,Mystery
366,366,Action
546,546,"Action, Thriller"
599,599,Romance
611,611,"Drama, Comedy"
771,771,"Romance, Western"
801,801,Drama


В данном подходе мы построили латентные матрицы и для пользователей, и для фильмов. Теперь можно использовать обе матрицы для предсказания рейтингов.

### 1.2.3 Generalized Matrix Factorization (GMF)

В данном подходе мы будем использовать нейросети для того, чтобы оценить латентные матрицы для пользователей и объектов.  
Затем, эти латентные матрицы можно будет использовать для предсказания.

Для нейросетей мы будем использовать фреймворк keras.
Мы будем использовать обертку над обычными моделями keras, которая сделана специально для построения рекомендательных систем – tensorflow-recommenders.

In [ ]:
#@title 
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

from typing import Dict, Text, Tuple

def df_to_ds(df):

    # convert pd.DataFrame to tf.data.Dataset
    ds = tf.data.Dataset.from_tensor_slices(
        (dict(df[['user_id','item_id']]), df['rating']))
    
    # convert Tuple[Dict[Text, tf.Tensor], tf.Tensor] to Dict[Text, tf.Tensor]
    ds = ds.map(lambda x, y: {
    'user_id' : x['user_id'],
    'item_id' : x['item_id'],
    'rating' : y
    })

    return ds.batch(256)

In [ ]:
class RankingModel(keras.Model):
    def __init__(self, user_id, item_id, embedding_size):
        super().__init__()
        
        # user model
        input = keras.Input(shape=(), dtype=tf.string)
        x = keras.layers.StringLookup(
            vocabulary = user_id, mask_token = None
            )(input)
        output = keras.layers.Embedding(
            input_dim = len(user_id) + 1,
            output_dim = embedding_size,
            name = 'embedding'
        )(x)
        self.user_model = keras.Model(inputs = input,
                                      outputs = output,
                                      name = 'user_model')

        # item model
        input = keras.Input(shape=(), dtype=tf.string)
        x = keras.layers.StringLookup(
            vocabulary = item_id, mask_token = None
            )(input)
        output = keras.layers.Embedding(
            input_dim = len(item_id) + 1,
            output_dim = embedding_size,
            name = 'embedding'
        )(x)
        self.item_model = keras.Model(inputs = input,
                                  outputs = output,
                                  name = 'item_model')

        # rating model
        user_input = keras.Input(shape=(embedding_size,), name='user_emb')
        item_input = keras.Input(shape=(embedding_size,), name='item_emb')
        x = keras.layers.Concatenate(axis=1)([user_input, item_input])
        x = keras.layers.Dense(256, activation = 'relu')(x)
        x = keras.layers.Dense(64, activation = 'relu')(x)
        output = keras.layers.Dense(1)(x)
        
        self.rating_model = keras.Model(
            inputs = {
                'user_id' : user_input,
                'item_id' : item_input
            },
            outputs = output,
            name = 'rating_model'
        )

    def call(self, inputs: Dict[Text, tf.Tensor]) -> tf.Tensor:
        user_emb = self.user_model(inputs['user_id'])
        item_emb = self.item_model(inputs['item_id'])

        prediction = self.rating_model({
            'user_id' : user_emb,
            'item_id' : item_emb
        })
        
        return prediction

class GMFModel(tfrs.models.Model):
    def __init__(self, user_id, item_id, embedding_size):
        super().__init__()
        self.ranking_model = RankingModel(user_id, item_id, embedding_size)
        self.task = tfrs.tasks.Ranking(
            loss = keras.losses.MeanSquaredError(),
            metrics = [keras.metrics.RootMeanSquaredError()]
        )
    
    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model(
            {
             'user_id' : features['user_id'], 
             'item_id' : features['item_id']
            })

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        return self.task(labels = features.pop('rating'),
                         predictions = self.ranking_model(features))

In [ ]:
train, test = train_test_split(ratings, train_size = .8, random_state=42)
train, test = df_to_ds(train), df_to_ds(test)

embedding_size = 64
model = GMFModel(user_id.astype(str),
                 item_id.astype(str),
                 embedding_size)
model.compile(
    optimizer = keras.optimizers.Adagrad(learning_rate = .01)
)

model.fit(train, epochs=3, verbose=0)

In [ ]:
result = model.evaluate(test, return_dict=True, verbose=0)
print("\nEvaluation on the test set:")
display(result)


Evaluation on the test set:


{'loss': 6.0332794189453125,
 'regularization_loss': 0,
 'root_mean_squared_error': 2.581247091293335,
 'total_loss': 6.0332794189453125}

In [ ]:
# extract item embeddings
item_emb = model.ranking_model.item_model.layers[-1].get_weights()[0]


item_corr_mat = cosine_similarity(item_emb)

print("\nThe top-k similar movie to item_id 99")
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)

display(items.loc[items[ITEM_COL].isin(similar_items)])


The top-k similar movie to item_id 99


,item_id,genres
97,97,Horror
99,99,"Romance, Action"
103,103,"Mystery, Romance"
234,234,"Western, Romance"
391,391,"Mystery, Drama"
642,642,Horror
714,714,Comedy
782,782,Thriller
784,784,"Mystery, Horror"
879,879,"Mystery, Romance"


# 2. Рекомендации с использованием текста

Настало время поработать с настоящими данными.  
Мы будем использовать датасет [Full MovieLens Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/data).

## 2.1 Popular-based

Для начала заново подготовим базовую модель для рекомендаций, но при этом будем использовать текст.

In [ ]:
!unzip "./movies_metadata.csv.zip" -d "./"

Archive:  ./movies_metadata.csv.zip
replace ./movies_metadata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd

metadata = pd.read_csv('/content/movies_metadata.csv', low_memory=False)
metadata = metadata[['id', 'overview', 'genres', 'title']]
metadata.head(3)

,id,overview,genres,title
0,862,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story
1,8844,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji
2,15602,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men


In [ ]:
metadata.shape

(45466, 24)

In [ ]:
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [ ]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [ ]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [ ]:
metadata.shape

(45466, 24)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(5)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385


Мы посчитали рейтинги фильмов по формуле IMDB и теперь можем сделать простую popularity-based рекомендательную систему, просто выбрав первые N элементов из таблицы выше.

## 2.2 Content-based

Теперь давайте попробуем усложнить модель, используя в качестве основы для рекомендаций описание фильма. Его можно получить из `metadata['overview']`

In [ ]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Для преобразования текста в числа мы будем пользоваться простым TF-IDF, хотя в проекте можно использовать и более сложные методы.

In [ ]:
# OOM here
metadata_small = metadata.iloc[:20000, :]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
metadata_small['overview'] = metadata_small['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(metadata_small['overview'])

tfidf_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(20000, 47487)

Теперь посчитаем сходства между разными описаниями фильмов.  
TfidfVectorizer нормализует строки, поэтому вместо `cosine_similarity` мы можем использовать `linear_kernel`

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(metadata_small.index, index=metadata_small['title']).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]

    return metadata_small['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight', cosine_sim)

18252                      The Dark Knight Rises
1328                              Batman Returns
15511                 Batman: Under the Red Hood
150                               Batman Forever
19792    Batman: The Dark Knight Returns, Part 1
585                                       Batman
3095                Batman: Mask of the Phantasm
18035                           Batman: Year One
9230          Batman Beyond: Return of the Joker
10122                              Batman Begins
Name: title, dtype: object

## 2.3. Улучшенная Content-based система

Усложним задачу: добавим новые данные из выборки.

In [ ]:
!unzip "./credits.csv.zip" -d "./"
!unzip "./keywords.csv.zip" -d "./"

Archive:  ./credits.csv.zip
replace ./credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  ./keywords.csv.zip
replace ./keywords.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

# Remove rows with bad IDs
metadata = metadata.drop([19730, 29503, 35587])

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [ ]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
import numpy as np

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [ ]:
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [ ]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
def create_combined_data(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' +\
     x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
metadata['combined_data'] = metadata.apply(create_combined_data, axis=1)

In [ ]:
metadata['combined_data'].head()

0    jealousy toy boy tomhanks timallen donrickles ...
1    boardgame disappearance basedonchildren'sbook ...
2    fishing bestfriend duringcreditsstinger walter...
3    basedonnovel interracialrelationship singlemot...
4    baby midlifecrisis confidence stevemartin dian...
Name: combined_data, dtype: object

Вместо TfidfVectorizer будем использовать CountVectorizer, так как мы не хотим снижать коэффициенты, например, у режиссеров, которые сняли много фильмов.

In [ ]:
metadata_small = metadata.iloc[:2680, :]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata_small['combined_data'])

In [ ]:
def cosine_similarity_n_space(m1, m2, batch_size=100):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    matrix_range = range(0, int(m1.shape[0] / batch_size) + 1)
    for row_i in matrix_range:
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break 
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) 
        ret[start: end] = sim
    return ret

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity_n_space(count_matrix, count_matrix, batch_size=10)

In [ ]:
get_recommendations('The Matrix', cosine_sim2)

730                   Ghost in the Shell
1216                      The Terminator
333                           Virtuosity
819                       Chain Reaction
1828                          Armageddon
2549    Superman IV: The Quest for Peace
374                              Timecop
2182                  Six-String Samurai
75                             Screamers
727                            Barb Wire
Name: title, dtype: object

# Рекомендации на основе корреляции

Теперь попробуем посмотреть на данные о рейтингах, которые пользователи давали фильмам. М

In [ ]:
metadata = pd.read_csv("./movies_metadata.csv")
metadata = metadata[['id', 'original_title', 'original_language']]
metadata = metadata.rename(columns={'id':'movieId'})
metadata = metadata[metadata['original_language']== 'en'] 
metadata.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,movieId,original_title,original_language
0,862,Toy Story,en
1,8844,Jumanji,en
2,15602,Grumpier Old Men,en
3,31357,Waiting to Exhale,en
4,11862,Father of the Bride Part II,en


In [ ]:
!unzip "./ratings.csv.zip" -d "./"

Archive:  ./ratings.csv.zip
  inflating: ./ratings.csv           


In [ ]:
ratings = pd.read_csv("./ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]

ratings = ratings.head(1000000)

In [ ]:
metadata.movieId = pd.to_numeric(metadata.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors= 'coerce')

In [ ]:
data = pd.merge(ratings, metadata, on='movieId', how='inner')
data.head()

,userId,movieId,rating,original_title,original_language
0,1,858,5.0,Sleepless in Seattle,en
1,3,858,4.0,Sleepless in Seattle,en
2,5,858,5.0,Sleepless in Seattle,en
3,12,858,4.0,Sleepless in Seattle,en
4,20,858,4.5,Sleepless in Seattle,en


In [ ]:
df = data.pivot_table(index='userId', columns='original_title', values='rating')
df.head()

original_title,!Women Art Revolution,$5 a Day,'Gator Bait,'R Xmas,'Twas the Night Before Christmas,...And the Pursuit of Happiness,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,15 Minutes,16 Blocks,1984,2 Days in Paris,2 Fast 2 Furious,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2001: A Space Travesty,2010,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,27 Dresses,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,3 días (Before the Fall),30 Beats,30 Days of Night,300,3:10 to Yuma,4 Little Girls,40 Days and 40 Nights,42nd Street,...,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Xtro,Yamakasi - Les samouraïs des temps modernes,Yankee Doodle Dandy,Yeast,You Are Not I,You Instead,You Kill Me,You Only Live Twice,You're a Big Boy Now,Young Adam,Young Black Stallion,Young Cassidy,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,Zaat,Zabriskie Point,Zandalee,Zapped Again!,Zardoz,Zathura: A Space Adventure,Zhenitba Balzaminova,"Zidane, un portrait du 21e siècle",Zodiac,Zombadings 1: Patayin sa Shokot si Remington,eXistenZ,xXx,¡Three Amigos!,Æon Flux,Бабник,Грозовые ворота,Дневник его жены,Мой сводный брат Франкенштейн,"Цирк сгорел, и клоуны разбежались",مارمولک,黑太陽731
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def pearson_R(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(movie_name, df, n):
    reviews = []
    for title in df.columns:
        if title == movie_name:
            continue
        cor = pearson_R(df[movie_name], df[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title, cor))
            
    reviews.sort(key = lambda tup: tup[1], reverse = True)
    return reviews[:n]

In [ ]:
recs = recommend('Harry Potter and the Goblet of Fire', df, 10) 
recs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


[('Flesh Gordon', 0.2785709413254664),
 ('Arrebato', 0.21310884596654495),
 ('Torn Curtain', 0.20792030084440874),
 ("Pirates of the Caribbean: At World's End", 0.19918632351891324),
 ('Paranoid Park', 0.19476482767591277),
 ('Lara Croft Tomb Raider: The Cradle of Life', 0.19232502605458207),
 ('Fuzz', 0.18944597645163191),
 ('The Longest Yard', 0.1835563932939968),
 ('Emma', 0.1824463306395699),
 ('SLC Punk', 0.17935979205608768)]